In [1]:
import numpy as np
import random

class AntColonyOptimizer:
    
    def __init__(self, num_ants, num_iterations, decay_factor, alpha, beta, q, pheromone_init):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay_factor = decay_factor
        self.alpha = alpha
        self.beta = beta
        self.q = q
        self.pheromone_init = pheromone_init
    
    def optimize(self, distance_matrix):
        num_cities = distance_matrix.shape[0]
        pheromone_matrix = np.full((num_cities, num_cities), self.pheromone_init)
        best_distance = float('inf')
        best_path = None
        
        for iteration in range(self.num_iterations):
            ant_paths = []
            ant_distances = []
            for ant in range(self.num_ants):
                path = self._build_path(pheromone_matrix, distance_matrix)
                ant_paths.append(path)
                distance = self._compute_distance(path, distance_matrix)
                ant_distances.append(distance)
                if distance < best_distance:
                    best_distance = distance
                    best_path = path
            
            pheromone_matrix *= self.decay_factor
            for path, distance in zip(ant_paths, ant_distances):
                pheromone_delta = self.q / distance
                for i in range(num_cities - 1):
                    j = i + 1
                    city_i = path[i]
                    city_j = path[j]
                    pheromone_matrix[city_i][city_j] += pheromone_delta
            
        return best_distance, best_path
    
    def _build_path(self, pheromone_matrix, distance_matrix):
        num_cities = distance_matrix.shape[0]
        visited_cities = set()
        current_city = random.randint(0, num_cities - 1)
        path = [current_city]
        visited_cities.add(current_city)
        
        while len(visited_cities) < num_cities:
            unvisited_cities = set(range(num_cities)) - visited_cities
            pheromone_values = [pheromone_matrix[current_city][j] ** self.alpha for j in unvisited_cities]
            distance_values = [distance_matrix[current_city][j] ** -self.beta for j in unvisited_cities]
            probabilities = np.multiply(pheromone_values, distance_values)
            probabilities /= np.sum(probabilities)
            next_city = np.random.choice(list(unvisited_cities), p=probabilities)
            path.append(next_city)
            visited_cities.add(next_city)
            current_city = next_city
        
        return path
    
    def _compute_distance(self, path, distance_matrix):
        distance = 0
        for i in range(len(path) - 1):
            j = i + 1
            city_i = path[i]
            city_j = path[j]
            distance += distance_matrix[city_i][city_j]
        return distance

In [2]:
num_cities = 10
distance_matrix = np.random.rand(num_cities, num_cities)

optimizer = AntColonyOptimizer(num_ants=2000000, num_iterations=100, decay_factor=0.5, alpha=1.0, beta=2.0, q=100, pheromone_init=0.1)

best_distance, best_path = optimizer.optimize(distance_matrix)

print("Best distance:", best_distance)
print("Best path:", best_path)

Best distance: 1.0631157371752793
Best path: [7, 1, 0, 4, 8, 2, 5, 3, 6, 9]
